In [ ]:
# %pip install ipython
%pip install --upgrade langchain -q

# requires for openai embedding
# %pip install tiktoken -q
# %pip install huggingface_hub
%pip install faiss-cpu
%pip install InstructorEmbedding
# %pip install sentence-transformers==2.2.2
# %pip install pypdf2
# %pip install ipywidgets

In [ ]:
from PIL import Image
from IPython.display import display

open_image = Image.open('/kaggle/input/cricket/flowchart.png')
display(open_image)

In [ ]:
from PyPDF2 import PdfReader

# loader = Docx2txtLoader('/content/drive/MyDrive/cricket_test/Cricket.docx')
# documents = loader.load()

def get_pdf_text(pdf_docs):
  text = ""
  pdf_reader = PdfReader(pdf_docs)
  for page in pdf_reader.pages:
    text += page.extract_text()
    
  return text

text = get_pdf_text('/kaggle/input/cricket/Cricket.pdf')
print(text)